In [3]:
#!pip install tensorflow
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
#If you want the Keras modules you write to be compatible with both 
#Theano (th) and TensorFlow (tf), you have to write them via the abstract Keras backend API.
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras.models import Model
#Till we have more data, we use ceil() at validation steps
import math

Using TensorFlow backend.


In [4]:
image_path="data"
IMG_SIZE=100
from keras.preprocessing.image import ImageDataGenerator
#https://keras.io/preprocessing/image/#imagedatagenerator-class
data_generator=ImageDataGenerator(validation_split=0.2)

train_data = data_generator.flow_from_directory(image_path,target_size=(100,100), color_mode='rgb', batch_size=10, subset='training')
valid_data = data_generator.flow_from_directory(image_path,target_size=(100,100), color_mode='rgb', batch_size=10, subset='validation')

STEP_SIZE = train_data.n//train_data.batch_size
VALIDATION_STEPS = math.ceil(valid_data.n / valid_data.batch_size)

print("Classes : ",train_data.class_indices)
print("VALIDATION_STEPS : ",VALIDATION_STEPS)

Found 1087 images belonging to 2 classes.
Found 270 images belonging to 2 classes.
Classes :  {'False_Images': 0, 'Processed_Images': 1}
VALIDATION_STEPS :  27


In [5]:
base_model=ResNet50(include_top=False, weights='imagenet')
x=base_model.output
x=GlobalAveragePooling2D()(x)

#we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x)
#x=Dropout(0.3)(x)
x=Dense(512,activation='relu')(x)
#x = Dense(256,activation='relu')(x)
#final layer with softmax activation
preds=Dense(2,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)
#model.summary()
print(len(model.layers))

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\DELL\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 786s 8us/step


AttributeError: module 'h5py' has no attribute 'File'

In [ ]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
    
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])
history = model.fit_generator(generator=train_data,
                   steps_per_epoch=STEP_SIZE,
                   validation_data=valid_data,
                   validation_steps=VALIDATION_STEPS,
                   epochs=15)

print(history.history.keys())

In [ ]:
#Plotting graphs for better viewing the model
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.show()

In [ ]:
model.summary()